In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from torchvision.transforms import v2
import sys

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


#### Loading the Dataset and normalization / transformation

In [ ]:
def get_transforms(random_crop=False, horizontal_flip=False,
                    translation=False, standardNormalize=False,
                      mu=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
    """
    Get transformation pipeline for training and test datasets.

    Args:
        random_crop (int): Padding size to apply random cropping.
        horizontal_flip (float): Probability to apply random horizontal flip.
        translation (tuple): Maximum absolute fraction for horizontal and vertical translations.
        standardNormalize (bool): Whether to apply normalization.
        mu (tuple): Mean for normalization.
        std (tuple): Standard deviation for normalization.

    Returns:
        torchvision.transforms.Compose: Transformation pipeline.
    """
    transform_list = []
    if random_crop:
        transform_list.append(v2.RandomCrop(random_crop, padding=4))
    if horizontal_flip:
        transform_list.append(v2.RandomHorizontalFlip(horizontal_flip))
    if translation:
        transform_list.append(v2.RandomAffine(degrees=0, translate=translation))
    
    transform_list.append(v2.ToTensor())
    
    if standardNormalize:
        transform_list.append(v2.Normalize(mu, std))
    
    return v2.Compose(transform_list)

In [ ]:
# mu = (0.4914, 0.4822, 0.4465)
# std=(0.2023, 0.1994, 0.2010)
#random_crop = 32 # crop size
#horizontal_flip = 0.5 # random flip probability
#translation = (0.1,0.1) # horizontal, vertical
batch_size = 64

train_transform = get_transforms(standardNormalize=True) # Parameters are defaulted to no change. change parameters as wanted
test_transform = get_transforms(standardNormalize=True)

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, pin_memory=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, pin_memory=True, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print('Train: X=%s, Y=(%s)' % (trainset.data.shape, len(trainset.targets)))
print('Test: X=%s, Y=(%s)' % (testset.data.shape, len(testset.targets)))

#### Test output size


$\text{Output\_size} = \left( \frac{\text{Input\_size} - \text{Kernel\_size} + 2 \times \text{Padding}}{\text{Stride}} \right) + 1$

In [ ]:
conv_layer = nn.Conv2d(3, 32, kernel_size=(3, 3), padding=1)

input_tensor = torch.randn(1, 3, 32, 32) # (batchsize, channels, height, width)

output_tensor = conv_layer(input_tensor)

print(output_tensor.shape)

#### Model functions

In [ ]:
def fit(model, dataLoader, criterion, optimizer, num_epochs):
    model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(dataLoader):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0


def evaluate_model(model, testloader):
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)  # Move data to GPU
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy on the test set: {100 * correct / total:.2f}%')

def create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0.0):
    return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


#### One Block VGG

In [ ]:
class Cifar10Model_1block(nn.Module):
    def __init__(self):
        super(Cifar10Model_1block,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(32*16*16, 128)
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.conv1(x)) # out = 32,32,32 (channels, height, width)
        x = self.relu(self.conv2(x)) # out = 32,32,32
        x = self.pool(x) # out = 32,16,16
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x)) # out = 128
        x = self.fc2(x) # out = 10
        return self.softmax(x)
    


In [ ]:
model = Cifar10Model_1block()
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
n_epochs = 10

In [ ]:
fit(model, train_loader, Loss, optimizer, n_epochs)

In [ ]:
evaluate_model(model, test_loader)

#### Three Block Baseline

In [ ]:
class Cifar10Model(nn.Module):
    def __init__(self):
        super(Cifar10Model,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), padding=1)

        self.conv3 = nn.Conv2d(32, 64, kernel_size=(3,3), padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=(3,3), padding=1)

        self.conv5 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=(3,3), padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.relu(self.conv1(x)) # out = 32,32,32 (channels, height, width)
        x = self.relu(self.conv2(x)) # out = 32,32,32
        x = self.pool(x) # out = 32,16,16
        
        x = self.relu(self.conv3(x)) # out = 64,16,16
        x = self.relu(self.conv4(x)) # out = 64,16,16
        x = self.pool(x) # out = 64,8,8
        
        x = self.relu(self.conv5(x)) # out = 128,8,8
        x = self.relu(self.conv6(x)) # out = 128,8,8
        x = self.pool(x) # out = 128,4,4
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x)) # out = 128
        x = self.fc2(x) # out = 10
        return self.softmax(x)

Inintialising model and it's params

In [ ]:
model = Cifar10Model()
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
n_epochs = 2

Training

In [ ]:
fit(model, train_loader, Loss, optimizer, n_epochs)

Evaluation

In [ ]:
evaluate_model(model, test_loader)

#### Full Baseline

In [ ]:
class Cifar10Model(nn.Module):
    def __init__(self, dropout_rate=0.0, batch_norm=False):
        super(Cifar10Model,self).__init__()
        self.batch_norm = batch_norm
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32) if batch_norm else nn.Identity()
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm2d(64) if batch_norm else nn.Identity()
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm2d(128) if batch_norm else nn.Identity()
        self.dropout3 = nn.Dropout(dropout_rate)
        
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.bn4 = nn.BatchNorm1d(128) if batch_norm else nn.Identity()
        self.dropout4 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))  # out = 32,32,32 (channels, height, width)
        x = self.relu(self.bn1(self.conv2(x)))  # out = 32,32,32
        x = self.pool1(x) # out = 32,16,16
        x = self.dropout1(x)
        
        x = self.relu(self.bn2(self.conv3(x)))  # out = 64,16,16
        x = self.relu(self.bn2(self.conv4(x))) # out = 64,16,16
        x = self.pool2(x) # out = 64,8,8
        x = self.dropout2(x)
        
        x = self.relu(self.bn3(self.conv5(x))) # out = 128,8,8
        x = self.relu(self.bn3(self.conv6(x))) # out = 128,8,8
        x = self.pool3(x) # out = 128,4,4
        x = self.dropout3(x)
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.bn4(self.fc1(x))) # out = 128
        x = self.dropout4(x)
        x = self.fc2(x) # out = 10
        return self.softmax(x)


#### Baseline + Dropout

In [ ]:
model = Cifar10Model(dropout_rate=0.2, batch_norm=False)
Loss = nn.CrossEntropyLoss()
optimizer = create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0)
n_epochs = 10

In [ ]:
fit(model, train_loader, Loss, optimizer, n_epochs)

In [ ]:
evaluate_model(model, test_loader)

#### Baseline + Weight Decay (L2)

In [ ]:
model = Cifar10Model(dropout_rate=0, batch_norm=False)
Loss = nn.CrossEntropyLoss()
optimizer = create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0.0005)
n_epochs = 10

In [ ]:
fit(model, train_loader, Loss, optimizer, n_epochs)

In [ ]:
evaluate_model(model, test_loader)

#### Baseline + Data Augmentation( horizontal flipping + translation)

Load data with augmentation settings

In [ ]:
# mu = (0.4914, 0.4822, 0.4465)
# std=(0.2023, 0.1994, 0.2010)
#random_crop = 32 # crop size
horizontal_flip = 0.5 # random flip probability
translation = (0.1,0.1) # horizontal, vertical
batch_size = 64

train_transform = get_transforms(standardNormalize=True,horizontal_flip = horizontal_flip, translation = translation) # Parameters are defaulted to no change. change parameters as wanted
test_transform = get_transforms(standardNormalize=True)

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, pin_memory=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, pin_memory=True, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [120]:
model = Cifar10Model(dropout_rate=0, batch_norm=False)
Loss = nn.CrossEntropyLoss()
optimizer = create_optimizer(model, lr=0.001, momentum=0.9, weight_decay=0.0000)
n_epochs = 10

In [ ]:
fit(model, train_loader, Loss, optimizer, n_epochs)

In [ ]:
evaluate_model(model, test_loader)